<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       SVM and SVMPredict Functions in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>SVM function is a linear support vector machine (SVM) that performs classification and regression analysis on datasets.This function supports these models:<ul  style = 'font-size:16px;font-family:Arial'>
    <li>Regression (loss: epsilon_insensitive).</li>
<li>Classification (loss: hinge). Only supports binary classification. The only response values are 0 or 1.</li></ul><p style = 'font-size:16px;font-family:Arial'> SVMPredict predicts target values (regression) and class labels (classification) for test data using an SVM model trained by SVM function.<br> In this notebook we will see how we can use the BincodeFit and BincodeTransform function available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_SVM_and_SVMPredict_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("teradataml", ["cal_housing_ex_raw"])

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame.from_table("cal_housing_ex_raw")
print("Shape of the data: ", tdf.shape)
tdf

<p style = 'font-size:16px;font-family:Arial'>Scale the data using Scalefit and ScaleTransform functions.</p>

In [ ]:
# Scale "target_columns" with respect to 'STD' value of the column.
fit_obj = ScaleFit(data=tdf,
                    target_columns=['MedInc', 'HouseAge', 'AveRooms',
                                    'AveBedrms', 'Population', 'AveOccup',
                                    'Latitude', 'Longitude'],
                    scale_method="STD")
 
 # Transform the data.
transform_obj = ScaleTransform(data=tdf,
                                object=fit_obj.output,
                                accumulate=["id", "MedHouseVal"])

<p style = 'font-size:16px;font-family:Arial'>Creating SVM model.<br>Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(SVM)

In [ ]:
# Train the transformed data using SVM when "model_type" is 'Regression'
svm_obj = SVM(data=transform_obj.result,
           input_columns=['MedInc', 'HouseAge', 'AveRooms',
                          'AveBedrms', 'Population', 'AveOccup',
                          'Latitude', 'Longitude'],
           response_column="MedHouseVal",
           model_type="Regression"
           ) 

In [ ]:
# Print the result DataFrame.
svm_obj.result

In [ ]:
svm_obj.output_data

<p style = 'font-size:16px;font-family:Arial'>Predict the values using SVMPredict function.<br>
Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(SVMPredict)

In [ ]:
SVMPredict_out = SVMPredict(newdata=transform_obj.result,
                             object=svm_obj.result,
                             id_column="id",
                             accumulate="MedHouseVal",
                             model_type="Regression"
                             )
  
# Print the result DataFrame.
SVMPredict_out.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
db_drop_table("cal_housing_ex_raw")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>SVM function reference: <a href = 'https://docs.teradata.com/search/all?query=SVM&content-lang=en-US'>here</a></li>
    <li>SVMPredict function reference: <a href = 'https://docs.teradata.com/search/all?query=SVMPredict&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>